### Student Information
Name: 蘇暐中

Student ID: 109020021

GitHub ID: SWCst1020575

Kaggle name (Group): 這是一個整人的活動嗎？笑死人喔？主辦單位，出來解釋一下好不好

Kaggle name (User): Wei-Chung Su

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the DM2023-Lab2-master. You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/09b1d0f3f8584d06848252277cb535f2. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 27th 11:59 pm, Wednesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 31th 11:59 pm, Sunday)__. 

In [1]:
### Begin Assignment Here

## Part 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_identification = pd.read_csv("data_identification.csv")
emotion = pd.read_csv("emotion.csv")
tweets_DM = pd.read_json("tweets_DM.json",lines=True)

In [4]:
tweets_DM[:5]

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets


In [5]:
print(tweets_DM["_index"].value_counts())
print(tweets_DM["_type"].value_counts())

_index
hashtag_tweets    1867535
Name: count, dtype: int64
_type
tweets    1867535
Name: count, dtype: int64


Which implies _index and _type have exact 1 type of value. Therefore, we drop these two columns.

In [6]:
tweets_drop = tweets_DM.drop(columns=["_index","_type"],axis=1)
tweets_drop[:5]

,_score,_source,_crawldate
0,391,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47
1,433,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09
2,232,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20
3,376,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05
4,989,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59


```json
{   // _source structure
    "tweet":{
        "hashtags":["tag","tag"],
        "tweet_id":"hash",
        "text": "post content ..."
    }
}
```

Extract dict from _source

In [7]:
hashtags = [ np.array(row['tweet']['hashtags']) for row in tweets_drop.values[:,1][:,] ]
tweet_id = [ row['tweet']['tweet_id'] for row in tweets_drop.values[:,1][:,]]
text = [ row['tweet']['text'] for row in tweets_drop.values[:,1][:,]]

In [8]:
tweets = pd.concat([tweets_drop,pd.Series(hashtags).to_frame("hashtags")
                    ,pd.Series(tweet_id).to_frame("tweet_id"),
                    pd.Series(text).to_frame("text")],axis=1)
tweets.drop(columns=["_source"],axis=1,inplace=True)


Remove @username, link and #tag

In [9]:
def textPreprocessing(processingData):
    text = processingData["text"]
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    while "<LH>" in new_text:
        new_text.remove("<LH>")
    new_text = " ".join(new_text)
    for tag in processingData["hashtags"]:
        new_text = new_text.replace(f"#{tag}","")
    processingData["text"] = new_text
    return processingData
# tweets["text"] = tweets["text"].apply(textPreprocessing)
tweets = tweets.apply(textPreprocessing,axis=1)


Combine data and split train and test.

In [10]:
tweets_identification = pd.merge(tweets,data_identification,on="tweet_id")
tweets_identification.drop("hashtags",axis=1,inplace=True)
tweets_train = tweets_identification[tweets_identification["identification"] == "train"]
tweets_test = tweets_identification[tweets_identification["identification"] == "test"]
tweets_train = pd.merge(tweets_train,emotion,on="tweet_id")
tweets_train.drop("identification",axis=1,inplace=True)
tweets_test.drop("identification",axis=1,inplace=True)

/tmp/ipykernel_18458/3920154719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_test.drop("identification",axis=1,inplace=True)


In [11]:
tweets_y = tweets_train["emotion"]
# tweets_train.drop("emotion",axis=1,inplace=True)

In [12]:
from datasets import Dataset
dataset = Dataset.from_pandas(tweets_train.drop(["tweet_id","_crawldate","_score"],axis=1))

/home/swc/DM/DM2023-Lab2-Master/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from sklearn.preprocessing import OneHotEncoder
emotionEncoder = OneHotEncoder()
emotionEncoder.fit(tweets_y.to_numpy().reshape(-1,1))
y = emotionEncoder.transform(tweets_y.to_numpy().reshape(-1,1))

### Model

In [14]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
from transformers import RobertaForSequenceClassification,RobertaTokenizerFast
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-emotion-latest', num_labels=8, ignore_mismatched_sizes=True).to(device)
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emotion-latest")

[2023-12-27 23:19:29,602] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion-latest and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def preprocess(data):
    newDict = {}
    text_tokenized = tokenizer(data["text"])
    labels = emotionEncoder.transform(np.array(data["emotion"]).reshape(-1,1))
    newDict["input_ids"] = text_tokenized["input_ids"]
    newDict["attention_mask"] = text_tokenized["attention_mask"]
    newDict["label"] = labels.toarray()
    return newDict
preprocessed = dataset.map(preprocess,batched=True)

Map:   0%|          | 0/1455563 [00:00<?, ? examples/s]

Map: 100%|██████████| 1455563/1455563 [00:22<00:00, 65221.25 examples/s]


In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
dataset = preprocessed.train_test_split(test_size=0.005,seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'emotion', 'input_ids', 'attention_mask', 'label'],
        num_rows: 1448285
    })
    test: Dataset({
        features: ['text', 'emotion', 'input_ids', 'attention_mask', 'label'],
        num_rows: 7278
    })
})

In [19]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids.argmax(-1)
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [20]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(
    output_dir='./results-latest-removeTag',
    num_train_epochs=3,
    learning_rate = 6e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=2500,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    metric_for_best_model="f1",
    save_steps=50000,
    eval_accumulation_steps=1,
    fp16=True,
    fp16_full_eval =True
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [32]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
2500,0.273800,0.256484,0.518686,0.498651
5000,0.256000,0.252908,0.538335,0.502553
7500,0.250300,0.248431,0.537785,0.512732
10000,0.247300,0.247769,0.547678,0.520419
12500,0.245400,0.245299,0.544518,0.511506
15000,0.241500,0.241737,0.550838,0.539969
17500,0.240600,0.238696,0.561968,0.540291
20000,0.238600,0.236003,0.565265,0.548345
22500,0.237000,0.241216,0.563479,0.543788
25000,0.237000,0.233707,0.571173,0.548562


KeyboardInterrupt: 

### Predict

In [21]:
def predPreprocess(data):
    newDict = {}
    text_tokenized = tokenizer(data["text"])
    newDict["input_ids"] = text_tokenized["input_ids"]
    newDict["attention_mask"] = text_tokenized["attention_mask"]
    return newDict

In [22]:
pred_dataset = Dataset.from_pandas(tweets_test.drop(["tweet_id","_crawldate","_score"],axis=1))
pred_preprocessed = pred_dataset.map(predPreprocess,batched=True)

Map: 100%|██████████| 411972/411972 [00:06<00:00, 59772.24 examples/s]


In [23]:
pred_model = RobertaForSequenceClassification.from_pretrained('./results-latest-removeTag/checkpoint-100000', num_labels=8, ignore_mismatched_sizes=True).to(device)
pred_trainer = Trainer(
        model=pred_model,
        args=training_args,
        train_dataset=None,
        eval_dataset=None,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

In [24]:
# predictions, labels, metrics = trainer.predict(pred_preprocessed)
predictions, labels, metrics = pred_trainer.predict(pred_preprocessed)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
predictions = torch.from_numpy(predictions)
pred_result = torch.argmax(predictions,dim=1).to(device)
pred_labels = torch.nn.functional.one_hot(pred_result)
pred_emotion = emotionEncoder.inverse_transform(pred_labels.cpu()).flatten()

In [26]:
id_emotion = np.concatenate([tweets_test["tweet_id"].to_numpy().reshape(-1,1),pred_emotion.reshape(-1,1)],axis=1)
id_emotion = pd.DataFrame(id_emotion,columns=["id","emotion"])
sample = pd.read_csv("sampleSubmission.csv").drop("emotion",axis=1)
output = pd.merge(sample,id_emotion,on="id")
output.to_csv("output.csv",index=False)